In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[1]:

etting the data

In [ ]:
import sagemaker
from sagemaker.local import LocalSession

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/Demo-pytorch-mnist"

In [ ]:
role = sagemaker.get_execution_role()

In[2]:

In [ ]:
role

In[3]:

In [ ]:
pip install torchvision==0.5.0 --no-cache-dir

In[4]:

In [ ]:
from torchvision import datasets, transforms

In [ ]:
datasets.MNIST(
    "data",
    download = True,
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    ),
)

In[5]:

In [ ]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print("input spec (in this case just an S3 path): {}".format(inputs))

In[8]:

In [ ]:
from sagemaker.pytorch import PyTorch

In [ ]:
estimator = PyTorch(
    entry_point = "mnist.py",
    role = role,
    framework_version = "1.4.0",
    py_version = "py3",
    instance_count = 2,
    instance_type = "ml.c4.xlarge",
    hyoerparameters = {"epochs": 6 , "backend": "gloo"}
)

In[9]:

In [ ]:
estimator.fit({"training": inputs})

In[10]:

In [ ]:
estimator.model_data

In[11]:

In [ ]:
predictor = estimator.deploy(initial_instance_count = 1, instance_type = "ml.t2.medium")

In[12]:

In [ ]:
get_ipython().system('ls data/MNIST/raw')

In[13]:

In [ ]:
import gzip
import numpy as np
import random
import os

In [ ]:
data_dir = 'data/MNIST/raw'
with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "rb") as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)
    
mask = random.sample(range(len(images)), 16)
mask = np.array(mask, dtype=np.int)
data = images[mask]

In[17]:

In [ ]:
response = predictor.predict(np.expand_dims(data, axis=1))
print('Raw prediction result:')
print(response)
print()

In [ ]:
labeled_predictions = list(zip(range(10), response[0]))
print('labeled_predictions:')
print(labeled_predictions)
print()

In [ ]:
labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most lokely answer: {}".format(labeled_predictions[0]))

In[19]:

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

In[23]:

In [ ]:
import numpy as np
image = np.array([data], dtype = np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

In[24]:

In [ ]:
predictor.delete_endpoint()

In[ ]: